In [93]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, f1_score, roc_auc_score
from catboost import CatBoostRegressor, CatBoostClassifier
from tabpfn import TabPFNClassifier

# Utils

In [94]:
import logging
import numpy as np
import os
import pandas as pd
import random
import tensorflow as tf
import torch
import yaml

def set_seed(seed: int = 42) -> None:
    """
    Set random seeds for reproducibility across Python, NumPy, PyTorch, and TensorFlow.

    Args:
        seed: Integer seed for random number generators
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    tf.random.set_seed(seed)

def setup_logging() -> logging.Logger:
    """
    Set up logging configuration.
    This function configures the logging settings for the application, including
    the logging level and format. It returns a logger instance that can be used
    throughout the application.
    Returns:
        logging.Logger: Configured logger instance.
    """
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    )
    logger = logging.getLogger(__name__)
    return logger


def check_GPU_availability():
    """
    Checks if a GPU is available and configures TensorFlow to use it appropriately.

    Sets up memory growth for TensorFlow GPU usage to avoid allocating all GPU memory at once.

    Returns:
        torch.device: A torch device object ('cuda' if GPU is available, 'cpu' otherwise)
    """
    logger = setup_logging()
    if torch.cuda.is_available():
        logger.info(f"Using GPU: {torch.cuda.get_device_name()}")
        device = torch.device("cuda")
        gpus = tf.config.experimental.list_physical_devices("GPU")
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    else:
        device = torch.device("cpu")
        logger.info("Using CPU for training.")
    return device

# Preprocessing

In [95]:
import numpy as np
import pandas as pd

# from common.utils import setup_logging
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import (
    StandardScaler,
    OneHotEncoder,
    LabelEncoder,
    TargetEncoder,
)


def encode_target(y):
    """
    Encode the target variable using LabelEncoder.

    Parameters
    ----------
    y : np.array
        The target variable to encode.

    Returns
    -------
    y_encoded : np.array
        The encoded target variable.
    """
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    return y_encoded


def preprocess(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_val: pd.DataFrame,
    cat_cols: list,
    config: dict,
    preprocessing_type: str,
) -> tuple:
    """
    Preprocess the training and validation datasets based on the specified model type.

    Parameters
    ----------
    X_train : pandas.DataFrame
        The training dataset features.
    y_train : pandas.Series or np.array
        The target variable for the training dataset.
    X_val : pandas.DataFrame
        The validation dataset features.
    cat_cols : list or array of bool
        Boolean mask indicating which columns in X_train and X_val are categorical (True).
    config : dict
        Configuration dictionary containing model and preprocessing details.

    Returns
    -------
    X_train : np.array
        The preprocessed training dataset features.
    X_val : np.array
        The preprocessed validation dataset features.
    """
    X_train, preprocessor_inner = _minimal_preprocess_train(
        X_train,
        y_train,
        cat_cols,
        preprocessing_type,
        config,
    )
    X_val = _minimal_preprocess_test(X_val, preprocessor_inner)

    return X_train, X_val


def _minimal_preprocess_train(X, y, categorical_features, model_type, config):
    """
    Perform minimal preprocessing on the input features for training.

    This function applies preprocessing to both numeric and categorical features
    based on the model type specified in the configuration. For Neural Networks,
    it standardizes numeric features and one-hot encodes categorical features. For
    tree-based models, it does not scale numeric features and applies different
    encoding strategies for low- and high-cardinality categorical features.

    Parameters
    ----------
    X : pandas.DataFrame
        The input features.

    categorical_features : list or array of bool
        Boolean mask indicating which columns in X are categorical (True) and
        which are numeric (False).

    Returns
    -------
    X_preprocessed : np.array
        The preprocessed features.
    preprocessor : ColumnTransformer
        The fitted preprocessor, which can be used to transform future datasets.
    """

    # Convert the boolean mask to a NumPy array (if not already) for fast indexing.
    cat_mask = np.asarray(categorical_features)

    # Check that the mask length matches the number of columns in X.
    if cat_mask.shape[0] != X.shape[1]:
        raise ValueError(
            "Length of categorical_features mask must match the number of columns in X."
        )

    # Extract column names using boolean indexing.
    categorical_feature_names = X.columns[cat_mask].tolist()
    numeric_feature_names = X.columns[~cat_mask].tolist()

    # Select pipeline based on config.
    logger = setup_logging()

    # Neural Networks: standardize numeric features and one-hot encode categoricals.
    if model_type == "nn":

        # Define the preprocessing pipeline for numeric features.
        numeric_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="mean")),
                ("scaler", StandardScaler()),
            ]
        )

        # Define the preprocessing pipeline for categorical features.
        categorical_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="most_frequent")),
                (
                    "onehot",
                    OneHotEncoder(
                        drop="first", sparse_output=False, handle_unknown="ignore"
                    ),
                ),
            ]
        )

        transformers = [
            ("num", numeric_pipeline, numeric_feature_names),
            ("cat", categorical_pipeline, categorical_feature_names),
        ]

        logger.info("Preprocessing pipeline for Neural Network model.")

    elif model_type == "tree":

        # Tree-based models: leave numeric features unscaled, and differentiate encoding for categoricals.
        numeric_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="mean")),
            ]
        )
        # Split categorical features into low- and high-cardinality groups.
        threshold = config["preprocessing"]["threshold_high_cardinality"]
        low_card_features = []
        high_card_features = []
        for feature in categorical_feature_names:
            if X[feature].nunique() <= threshold:
                low_card_features.append(feature)
            else:
                high_card_features.append(feature)

        # For low-cardinality categoricals, use one-hot encoding.
        low_card_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="most_frequent")),
                (
                    "onehot",
                    OneHotEncoder(
                        drop="first", sparse_output=False, handle_unknown="ignore"
                    ),
                ),
            ]
        )

        # For high-cardinality categoricals, use target encoding.
        high_card_pipeline = Pipeline(
            [
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("target_encode", TargetEncoder()),
            ]
        )

        transformers = [
            ("num", numeric_pipeline, numeric_feature_names),
            ("low_card", low_card_pipeline, low_card_features),
            ("high_card", high_card_pipeline, high_card_features),
        ]

        logger.info("Preprocessing pipeline for Decision Tree model.")

    elif model_type == "grande":
        logger.info("No preprocessing required for Grande model.")
        return X, None

    else:
        raise ValueError(
            "Unknown teacher type. Must be 'nn' for Neural Network, 'tree' for Decision Tree, or 'grande' for Gradient Based Tree."
        )

    # Create a ColumnTransformer to apply the transformations to the appropriate columns.
    preprocessor = ColumnTransformer(
        transformers=transformers,
        n_jobs=-2,
    )

    # Fit the preprocessor on the training data and transform it.
    X_preprocessed = preprocessor.fit_transform(X, y)

    return X_preprocessed, preprocessor


def _minimal_preprocess_test(X, preprocessor):
    """
    Transform the input features according to the preprocessor fitted on the training data.

    Parameters
    ----------
    X : pandas.DataFrame
        The input features.
    preprocessor : ColumnTransformer
        The preprocessor fitted on the training data.

    Returns
    -------
    X_preprocessed : np.array
        The preprocessed features.
    """
    if preprocessor is None:
        return X
    return preprocessor.transform(X)


# Data Loader

In [96]:
import numpy as np
import openml
import os
import pandas as pd
import pickle
# from common.preprocessing import encode_target
# from common.utils import setup_logging
from typing import Literal

def load_dataset(dataset_id: int, config: dict):
    """
    Loads an OpenML dataset based on its ID and processes it according to the task type.

    This function fetches the dataset using the _fetch_dataset helper, then processes
    the target variable based on whether the task is binary classification or regression.

    Parameters:
        dataset_id (int): The ID of the dataset on OpenML.
        config (dict): Configuration dictionary containing data paths and settings.

    Returns:
        tuple: A tuple containing:
            - X: The feature data (pandas DataFrame)
            - y: The processed target variable (encoded for binary classification or numpy array for regression)
            - cat_cols: List of booleans indicating which features are categorical
            - attribute_names: List of attribute names
            - task_type: String indicating the task type ('binary' or 'regression')
    """
    X, y, cat_cols, attribute_names, task_type = fetch_dataset(
        dataset_id=dataset_id,
        cache_dir=config["data"]["cache_dir_path"],
    )
    if task_type == "binary":
        # Encode the target variable if it's binary
        y = encode_target(y)
    else:
        # Transform to np.array for regression
        y = np.array(y, dtype=float)

    return X, y, cat_cols, attribute_names, task_type


def fetch_dataset(dataset_id: int, cache_dir: str = "data/cache/"):
    """
    Downloads an OpenML dataset based on its id, caches the data locally, and returns the data
    along with its metadata.

    The five returned objects are:
        - X: The feature data (typically a pandas DataFrame).
        - y: The target variable.
        - categorical_indicator: A list of booleans indicating which features are categorical.
        - attribute_names: A list of attribute names.
        - task_type: A string indicating the type of task ('binary' or 'regression').

    If the dataset has been previously downloaded and stored in the cache directory,
    it will be loaded from the local file instead of re-downloading.

    Parameters:
        dataset_id (int): The id of the dataset on OpenML.
        cache_dir (str): The directory to store the downloaded dataset. Defaults to "openml_cache".

    Returns:
        X, y, categorical_indicator, attribute_names, task_type
    """
    logger = setup_logging()

    # Ensure the cache directory exists
    if not os.path.exists(cache_dir):
        os.makedirs(cache_dir, exist_ok=True)

    # Define a cache file name that is unique to the dataset id
    cache_file = os.path.join(cache_dir, f"openml_dataset_{dataset_id}.pkl")

    # If the cache file exists, load the data from the file.
    if os.path.exists(cache_file):
        logger.info(
            f"Loading dataset {dataset_id} from cache at '{cache_file}'..."
        )
        with open(cache_file, "rb") as f:
            data = pickle.load(f)
        X = data["X"]
        y = data["y"]
        categorical_indicator = data["categorical_indicator"]
        attribute_names = data["attribute_names"]
        task_type = data["task_type"]
    else:
        # Download the dataset from OpenML.
        logger.info(f"Downloading dataset {dataset_id} from OpenML...")
        dataset = openml.datasets.get_dataset(dataset_id)

        # Use the default target attribute (if defined) when retrieving the data.
        X, y, categorical_indicator, attribute_names = dataset.get_data(
            target=dataset.default_target_attribute, dataset_format="dataframe"
        )

        # Determine the task type (binary classification or regression)
        task_type = _determine_task_type(y)

        # Store the data and metadata in a dictionary.
        data = {
            "X": X,
            "y": y,
            "categorical_indicator": categorical_indicator,
            "attribute_names": attribute_names,
            "task_type": task_type,
        }

        # Save the dictionary to a local file using pickle.
        with open(cache_file, "wb") as f:
            pickle.dump(data, f)
        logger.info(f"Dataset {dataset_id} stored locally at '{cache_file}'.")

    logger.info(f"Dataset {dataset_id} loaded successfully with task type: {task_type}")

    return X, y, categorical_indicator, attribute_names, task_type


def _determine_task_type(y):
    """
    Determines if the target variable is for binary classification, or regression.

    Parameters:
        y: The target variable (pandas dataframe).

    Returns:
        str: 'binary', or 'regression'
    """

    # Get unique values
    unique_values = pd.unique(y)

    # Check if it's binary (2 unique values)
    if len(unique_values) == 2:
        return "binary"
    else:
        return "regression"

# Evaluate

In [97]:
import numpy as np
from sklearn.metrics import (
    balanced_accuracy_score,
    f1_score,
    roc_auc_score,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)

def evaluate_classification(y_prob, y_true):
    """
    Evaluate classification performance using balanced accuracy, F1 score, and ROC AUC.

    Parameters:
        y_prob (np.array): Predicted probabilities for the positive class.
        y_true (np.array): True labels.

    Returns:
        dict: Dictionary containing evaluation metrics.
    """
    logger = setup_logging()
    y_pred = (y_prob[:, 1] > 0.5).astype(int)

    acc = balanced_accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average="macro")
    roc_auc = roc_auc_score(y_true, y_prob[:, 1])

    logger.info(f"\t Balanced Accuracy: {acc:.4f}, F1 Score: {f1:.4f}, ROC AUC: {roc_auc:.4f}")

    return {
        "acc": acc,
        "f1": f1,
        "roc": roc_auc,
    }

def evaluate_regression(y_pred, y_true):
    """
    Evaluate regression performance using R^2 score.

    Parameters:
        y_pred (np.array): Predicted values.
        y_true (np.array): True values.

    Returns:
        float: R^2 score.
    """
    logger = setup_logging()

    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)

    logger.info(f"\t MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R^2: {r2:.4f}")

    return {
        "mae": mae,
        "mse": mse,
        "rmse": rmse,
        "r2": r2,
    }


# Model Factory

In [98]:
import numpy as np
import torch
import torch.nn.functional as F

from abc import ABC, abstractmethod
from catboost import CatBoostClassifier, CatBoostRegressor
# from common.packages.tabm_reference import Model, make_parameter_groups
from tabpfn import TabPFNClassifier, TabPFNRegressor
from GRANDE import GRANDE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.model_selection import train_test_split
from typing import Dict


class TeacherModelBase(ABC):
    def __init__(self, **kwargs):
        self.params = kwargs

    @abstractmethod
    def train(self, X: np.ndarray, y: np.ndarray) -> None:
        """
        Trains the model on the given data.

        Parameters:
            X (np.ndarray): The input features with shape (n_samples, n_features).
            y (np.ndarray): The target labels with shape (n_samples,).

        Returns:
            None
        """
        pass

    @abstractmethod
    def predict(self, X: np.ndarray) -> np.ndarray:
        """
        Makes predictions on the provided data.

        Parameters:
            X (np.ndarray): The input features with shape (n_samples, n_features).

        Returns:
            np.ndarray: An array of predicted probabilities with shape (n_samples, n_classes).
                        (If the model returns logits, these should be post-processed to probabilities.)
        """
        pass

    def evaluate(self, y_pred: np.ndarray, y_true: np.ndarray) -> Dict[str, float]:
        """
        Evaluates the model's predictions against the true labels.

        Parameters:
            y_pred (np.ndarray): The predicted probabilities or values.
            y_true (np.ndarray): The true labels.

        Returns:
            Dict[str, float]: A dictionary containing evaluation metrics.
        """
        if self.task_type == "binary":
            return evaluate_classification(y_pred, y_true)
        elif self.task_type == "regression":
            return evaluate_regression(y_pred, y_true)
        else:
            raise ValueError(f"Unknown task type: {self.task_type}. Must be 'binary' or 'regression'.")

    @abstractmethod
    def count_parameters(self) -> int:
        """
        Counts the number of trainable parameters in the model.

        Returns:
            int: The total number of trainable parameters.
        """
        pass


class TabPFNTeacherModel(TeacherModelBase):

    def __init__(self, **kwargs):
        """
        Initialize a TabPFN teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base
            class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        super().__init__(**kwargs)

    def train(self, X, y, **training_params):
        if self.task_type == "binary":
            self.model = TabPFNClassifier()
        else:
            self.model = TabPFNRegressor()
        # If X is bigger than 10000 samples, reduce data size to 10000
        if X.shape[0] > 10000:
            X = X[:10000]
            y = y[:10000]
        self.model.fit(X, y)

    def predict(self, X):
        if self.task_type == "binary":
            return self.model.predict_proba(X)
        else:
            return self.model.predict(X)

    def count_parameters(self) -> int:
        """
        Counts the number of trainable parameters in the underlying TabPFN model.

        Returns:
            int: Total number of trainable parameters.

        Raises:
            ValueError: If the TabPFN model has not been fitted yet.
        """
        if not hasattr(self.model, "model_"):
            raise ValueError(
                "The TabPFN model is not fitted yet. Please call fit() first."
            )
        return sum(p.numel() for p in self.model.model_.parameters() if p.requires_grad)
    
class CatBoostTeacherModel(TeacherModelBase):
    def __init__(self, **kwargs):
        """
        Initialize a CatBoost teacher model.

        Parameters
        ----------
        **kwargs : dict
            Additional keyword arguments to pass to the base class constructor.
        """
        self.device = kwargs.pop("device")
        self.config = kwargs.pop("config")
        self.task_type = kwargs.pop("task_type")
        super().__init__(**kwargs)

    def train(self, X, y, **training_params):
        """
        Train the CatBoost model using the provided data.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.
        y : array-like of shape (n_samples,)
            The target data.
        **training_params : dict
            Additional training parameters to pass to the CatBoost model's fit method.
        """
        config = self.config
        random_state = config["training"]["random_state"]

        # Initialize the CatBoost model
        if self.task_type == "binary":
            self.model = CatBoostClassifier(
                random_seed=random_state,
                thread_count=-1,  # Use all available CPU cores
                logging_level="Silent",  # Suppress CatBoost output
                task_type=("GPU" if str(self.device).lower() == "cuda" else "CPU"),
                **self.params,  # Include the sampled hyperparameters
            )
        else:
            self.model = CatBoostRegressor(
                random_seed=random_state,
                thread_count=-1,  # Use all available CPU cores
                logging_level="Silent",  # Suppress CatBoost output
                task_type=("GPU" if str(self.device).lower() == "cuda" else "CPU"),
                **self.params,  # Include the sampled hyperparameters
            )

        # Split data into training and validation sets
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=random_state
        )

        # Train the model
        self.model.fit(
            X_train,
            y_train,
            eval_set=(X_val, y_val),
            **training_params,  # Pass additional training parameters
        )

    def predict(self, X):
        """
        Make predictions on the given data using the trained model.

        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            The input data.

        Returns
        -------
        array-like of shape (n_samples, n_classes)
            The predicted probabilities for each class.
        """
        if self.task_type == "binary":
            return self.model.predict_proba(X)
        else:
            return self.model.predict(X)

    def count_parameters(self):
        """
        Count the number of parameters in the trained CatBoost model.

        Returns
        -------
        int
            The number of parameters in the model.
        """
        if not hasattr(self, "model") or self.model is None:
            raise ValueError("The model has not been trained yet.")

        # Get the number of trees in the model
        tree_count = self.model.tree_count_

        # For each tree, count:
        # - split values (one per non-leaf node)
        # - leaf values (one per leaf node)
        # - feature indices (one per non-leaf node)
        # For a binary tree with depth d, there are 2^d - 1 non-leaf nodes and 2^d leaf nodes
        # This is a simplified approximation
        approx_depth = self.params.get("depth", 6)  # Default depth in CatBoost is 6
        non_leaf_nodes = 2**approx_depth - 1
        leaf_nodes = 2**approx_depth

        # Total parameters per tree
        params_per_tree = (
            non_leaf_nodes * 2 + leaf_nodes
        )  # split values + feature indices + leaf values

        return tree_count * params_per_tree
    
def get_teacher_model(config: dict, task_type: Literal["binary", "regression"], device) -> TeacherModelBase:
    """
    Returns an instance of the teacher model based on the configuration.

    Parameters
    ----------
    config : dict
        Configuration dictionary containing model and dataset details.
    device : torch.device
        The device to use for training (CPU or GPU).

    Returns
    -------
    TeacherModelBase
        An instance of the teacher model.
    """
    model_type = config["model"]["teacher_model"]
    if model_type == "tabpfn":
        return TabPFNTeacherModel(config=config, task_type=task_type, device=device)
    elif model_type == "catboost":
        return CatBoostTeacherModel(config=config, task_type=task_type, device=device)
    else:
        raise ValueError(f"Unknown teacher type: {config['model']['teacher_model']}")

# Config

In [99]:
config = {
    "data": {
        "datasets": [
                     23381, # Binary classification datasets
                    #    197, # Regression datasets
                     ],  
        "cache_dir_path": "data/cache/",
    },

    "preprocessing": {
        "threshold_high_cardinality": 10,  
    },

    "model": {
        "teacher_model": "tabpfn",   # Options: 'tabpfn', 
        "student_model": "catboost", # Options: 'catboost', 
    },

    "training": {
        "random_state": 42,
        "outer_folds": 5,
        "inner_folds": 2,
    },

    "teacher_models": {
        "tabpfn": {
            "preprocessing": "nn",  
        },
        "catboost": {
            "preprocessing": "tree", 
        },
    },
}

# Train Teacher

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
import time

# Get list of datasets to process from configuration
datasets = config["data"]["datasets"]

# =============================================================================
# MAIN TRAINING LOOP - Process each dataset independently
# =============================================================================
for dataset_id in datasets:
# -------------------------------------------------------------------------
    # SETUP AND INITIALIZATION
    # -------------------------------------------------------------------------
    logger = setup_logging()
    logger.info("Loading configuration...")

    # Extract model configuration for this run
    model_type = config["model"]["teacher_model"]
    preprocessing_type = config["teacher_models"][model_type]["preprocessing"] 

    # -------------------------------------------------------------------------
    # STEP 0: DATA LOADING
    # -------------------------------------------------------------------------
    # Load dataset from OpenML with caching for efficiency
    X, y, cat_cols, _, task_type = load_dataset(
        dataset_id=dataset_id,
        config=config,
    )

    # -------------------------------------------------------------------------
    # STEP 1: INFRASTRUCTURE SETUP
    # -------------------------------------------------------------------------
    # Note: Checking for existing results is placeholder for future implementation

    # Configure GPU/CPU usage for training
    device = check_GPU_availability()

    # Set random seed for reproducibility across all libraries
    set_seed(config["training"]["random_state"])
    logger.info(f"Random seed set to {config['training']['random_state']}")

    # -------------------------------------------------------------------------
    # STEP 2: INITIALIZE DATA STRUCTURES
    # -------------------------------------------------------------------------
    # List to store predictions from each outer fold
    output_dfs = []
    
    # Dictionary to store all fold indices for reproducibility and student training
    # Structure: {"outer_folds": {fold_id: {train_idx, test_idx}},
    #            "inner_folds": {outer_fold_id: {inner_fold_id: {train_idx, val_idx}}}}
    fold_indices = {
        "outer_folds": {},
        "inner_folds": {}
    }

    # -------------------------------------------------------------------------
    # STEP 3: OUTER CROSS-VALIDATION SETUP
    # -------------------------------------------------------------------------
    # Choose appropriate CV strategy based on task type to maintain class balance
    if task_type == "binary":
        outer_cv = StratifiedKFold(
            n_splits=config["training"]["outer_folds"],
            shuffle=True,
            random_state=config["training"]["random_state"],
        )
    else:
        outer_cv = KFold(
            n_splits=config["training"]["outer_folds"],
            shuffle=True,
            random_state=config["training"]["random_state"],
        )

    # =========================================================================
    # STEP 4: OUTER CROSS-VALIDATION LOOP
    # =========================================================================
    # Each iteration provides one unbiased performance estimate
    for outer_fold, (train_idx, test_idx) in enumerate(outer_cv.split(X, y), start=1):

        logger.info(f"-------------------- Outer Fold {outer_fold} --------------------")
        
        # ---------------------------------------------------------------------
        # FOLD INDEX MANAGEMENT
        # ---------------------------------------------------------------------
        # Store outer fold indices for later use in student training and validation
        fold_indices["outer_folds"][f"fold_{outer_fold}"] = {
            "train_idx": train_idx.tolist(),
            "test_idx": test_idx.tolist()
        }

        # Split data according to current outer fold
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # ---------------------------------------------------------------------
        # INNER CROSS-VALIDATION SETUP (for model validation)
        # ---------------------------------------------------------------------
        # Choose appropriate CV strategy for inner folds
        if task_type == "binary":
            inner_cv = StratifiedKFold(
                n_splits=config["training"]["inner_folds"],
                shuffle=True,
                random_state=config["training"]["random_state"],
            )
        else:
            inner_cv = KFold(
                n_splits=config["training"]["inner_folds"],
                shuffle=True,
                random_state=config["training"]["random_state"],
            )
        
        # Initialize storage for inner fold indices within this outer fold
        fold_indices["inner_folds"][f"outer_fold_{outer_fold}"] = {}

        # =====================================================================
        # STEP 5: INNER CROSS-VALIDATION LOOP (hyperparameter validation)
        # =====================================================================
        # This loop would typically be used for hyperparameter optimization
        for inner_fold, (inner_train_index, inner_val_index) in enumerate(inner_cv.split(X_train, y_train), start=1):
            
            # -----------------------------------------------------------------
            # INDEX MANAGEMENT (Critical for avoiding data leakage)
            # -----------------------------------------------------------------
            # Convert relative indices (within outer training set) to absolute indices
            # This is crucial for maintaining consistency with teacher outputs
            absolute_inner_train_idx = train_idx[inner_train_index]
            absolute_inner_val_idx = train_idx[inner_val_index]
            
            # Store inner fold indices using absolute indices for consistency
            fold_indices["inner_folds"][f"outer_fold_{outer_fold}"][f"inner_fold_{inner_fold}"] = {
                "train_idx": absolute_inner_train_idx.tolist(),
                "val_idx": absolute_inner_val_idx.tolist()
            }

            # Split inner training data using relative indices
            X_inner_train, X_inner_val = X_train.iloc[inner_train_index], X_train.iloc[inner_val_index]
            y_inner_train, y_inner_val = y_train[inner_train_index], y_train[inner_val_index]

            # -----------------------------------------------------------------
            # PREPROCESSING: Apply model-specific data transformations
            # -----------------------------------------------------------------
            X_inner_train, X_inner_val = preprocess(
                X_inner_train,
                y_inner_train,
                X_inner_val, 
                cat_cols,
                config,
                preprocessing_type=preprocessing_type,
            )

            # -----------------------------------------------------------------
            # MODEL TRAINING: Train teacher model on inner training data
            # -----------------------------------------------------------------
            model = get_teacher_model(config=config, task_type=task_type, device=device)
            logger.info(f"Training Model on Outer Fold {outer_fold}, Inner Fold {inner_fold}...")
            model.train(X_inner_train, y_inner_train)

            # -----------------------------------------------------------------
            # VALIDATION: Evaluate model performance on inner validation set
            # -----------------------------------------------------------------
            val_preds = model.predict(X_inner_val)
            val_metrics = model.evaluate(val_preds, y_inner_val)

        # =====================================================================
        # STEP 6: FINAL MODEL TRAINING (on complete outer training set)
        # =====================================================================
        # After inner CV completes, train final model on all available training data
        
        # Apply same preprocessing to outer training and test sets
        X_train, X_test = preprocess(
            X_train,
            y_train,
            X_test,
            cat_cols,
            config,
            preprocessing_type=preprocessing_type,  # Use 'nn' for TabPFN preprocessing
        )
        
        # Train final model on complete outer training set
        logger.info("------------------------------------------------------")
        logger.info(f"Retraining Model on Outer Fold {outer_fold}")
        model.train(X_train, y_train)

        # =====================================================================
        # STEP 7: FINAL EVALUATION (unbiased performance on outer test set)
        # =====================================================================
        # This provides the unbiased performance estimate for this fold
        start_time = time.time()
        test_preds = model.predict(X_test)
        end_time = time.time() - start_time
        logger.info(f"\t Inference Time: {end_time:.5f} seconds")
        test_metrics = model.evaluate(test_preds, y_test)

        # =====================================================================
        # STEP 8: STORE PREDICTIONS FOR STUDENT TRAINING
        # =====================================================================
        # Save predictions with their corresponding dataset indices
        # These will be used as targets for training student models
        output_dfs.append(pd.DataFrame({
            "index": test_idx,
            "output": test_preds[:, 1] if task_type == "binary" else test_preds
        }))
    
    # =========================================================================
    # STEP 9: SAVE RESULTS AND METADATA
    # =========================================================================
    
    # -------------------------------------------------------------------------
    # SAVE FOLD INDICES (for reproducibility and student training)
    # -------------------------------------------------------------------------
    import json
    fold_indices_file = os.path.join(config["data"]["cache_dir_path"], f"dataset_{dataset_id}_fold_indices.json")
    with open(fold_indices_file, 'w') as f:
        json.dump(fold_indices, f, indent=2)
    print(f"Fold indices saved to: {fold_indices_file}")

    # -------------------------------------------------------------------------
    # SAVE TEACHER PREDICTIONS (targets for student training)
    # -------------------------------------------------------------------------
    # Combine predictions from all outer folds
    if output_dfs:  # Check if we have any DataFrames to concatenate
        output_df = pd.concat(output_dfs, ignore_index=True)
    else:
        output_df = pd.DataFrame(columns=["index", "output"])
    output_df = output_df.sort_values(by="index")
    output_file = os.path.join(config["data"]["cache_dir_path"], f"teacher_outputs_{dataset_id}_{model_type}.csv")
    output_df.to_csv(output_file, index=False)
    print(f"{config['model']['teacher_model']} outputs saved to: {output_file}")

2025-06-04 11:04:29,824 - __main__ - INFO - Loading configuration...
2025-06-04 11:04:29,827 - __main__ - INFO - Loading dataset 23381 from cache at 'data/cache/openml_dataset_23381.pkl'...
2025-06-04 11:04:29,835 - __main__ - INFO - Dataset 23381 loaded successfully with task type: binary
2025-06-04 11:04:29,840 - __main__ - INFO - Using GPU: NVIDIA RTX A6000
2025-06-04 11:04:29,845 - __main__ - INFO - Random seed set to 42
2025-06-04 11:04:29,855 - __main__ - INFO - -------------------- Outer Fold 1 --------------------


2025-06-04 11:04:29,865 - __main__ - INFO - Preprocessing pipeline for Neural Network model.
/home/mherre/miniconda3/envs/thesis/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 2, 3, 4, 5, 6, 7, 8, 9, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
2025-06-04 11:04:35,040 - __main__ - INFO - Training Model on Outer Fold 1, Inner Fold 1...
2025-06-04 11:04:36,403 - __main__ - INFO - 	 Balanced Accuracy: 0.5837, F1 Score: 0.5766, ROC AUC: 0.6439
2025-06-04 11:04:36,418 - __main__ - INFO - Preprocessing pipeline for Neural Network model.
/home/mherre/miniconda3/envs/thesis/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [0, 1, 4, 5, 7, 8, 9, 10] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
2025-06-04 11:04:40,602 - __main__ - INFO - Training Model on Outer Fold 

Fold indices saved to: data/cache/dataset_23381_fold_indices.json
tabpfn outputs saved to: data/cache/teacher_outputs_23381_tabpfn.csv


# Train Student

In [ ]:
from sklearn.model_selection import StratifiedKFold, KFold
import json

datasets = config["data"]["datasets"]

for dataset_id in datasets:
    X, y, cat_cols, _, task_type = load_dataset(
        dataset_id=dataset_id,
        config=config,
    )

    device = check_GPU_availability()

    set_seed(config["training"]["random_state"])

    # Load the saved fold indices 
    fold_indices_file = os.path.join(config["data"]["cache_dir_path"], f"dataset_{dataset_id}_fold_indices.json")
    with open(fold_indices_file, 'r') as f:
        fold_indices = json.load(f)
    print(f"Loaded fold indices from: {fold_indices_file}") 

    # Load the TabPFN outputs (teacher predictions)
    tabpfn_outputs_file = os.path.join(config["data"]["cache_dir_path"], f"dataset_{dataset_id}_tabpfn_outputs.csv")
    teacher_outputs_df = pd.read_csv(tabpfn_outputs_file)
    print(f"Loaded TabPFN outputs from: {tabpfn_outputs_file}")

    # Convert probabilities to logits
    # Clip probabilities to avoid log(0) or log(1)
    eps = 1e-7
    teacher_probs = np.clip(teacher_outputs_df['output'].values, eps, 1 - eps)
    teacher_logits = np.log(teacher_probs / (1 - teacher_probs))
    
    # Create a mapping from index to logits for easy lookup
    index_to_logits = dict(zip(teacher_outputs_df['index'].values, teacher_logits))

    output_df = pd.DataFrame(columns=["index", "output"])

    for fold_key, fold_data in fold_indices["outer_folds"].items():
        outer_fold = int(fold_key.split('_')[1])
        train_idx = np.array(fold_data["train_idx"])
        test_idx = np.array(fold_data["test_idx"])

        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        inner_folds_data = fold_indices["inner_folds"][f"outer_fold_{outer_fold}"]

        for inner_fold_key, inner_fold_data in inner_folds_data.items():
            inner_fold = int(inner_fold_key.split('_')[2])
            
            # Get absolute indices from saved data
            absolute_inner_train_idx = np.array(inner_fold_data["train_idx"])
            absolute_inner_val_idx = np.array(inner_fold_data["val_idx"])
            
            # Convert absolute indices to relative indices for the current outer training set
            inner_train_relative = np.where(np.isin(train_idx, absolute_inner_train_idx))[0]
            inner_val_relative = np.where(np.isin(train_idx, absolute_inner_val_idx))[0]

            X_inner_train, X_inner_val = X_train.iloc[inner_train_relative], X_train.iloc[inner_val_relative]
            y_inner_train, y_inner_val = y_train[inner_train_relative], y_train[inner_val_relative]

            # Get teacher logits for inner training and validation sets
            teacher_logits_inner_train = np.array([index_to_logits[idx] for idx in absolute_inner_train_idx])
            teacher_logits_inner_val = np.array([index_to_logits[idx] for idx in absolute_inner_val_idx])

            # Preprocess the data for tree-based model
            X_inner_train, X_inner_val = preprocess(
                X_inner_train,
                y_inner_train,
                X_inner_val, 
                cat_cols,
                config,
                preprocessing_type="tree",
            )

            # Train CatBoost regressor to predict teacher logits
            catboost = CatBoostRegressor(verbose=False)
            catboost.fit(X_inner_train, teacher_logits_inner_train)

            # Evaluate on validation set
            val_pred_logits = catboost.predict(X_inner_val)
            # Convert predicted logits back to probabilities for evaluation
            val_probs = 1 / (1 + np.exp(-val_pred_logits))
            val_preds = (val_probs > 0.5).astype(int)
            
            val_acc = balanced_accuracy_score(y_inner_val, val_preds)
            val_f1 = f1_score(y_inner_val, val_preds, average="macro")
            val_roc_auc = roc_auc_score(y_inner_val, val_probs)
            print(f"Outer Fold {outer_fold}, Inner Fold {inner_fold} - Accuracy: {val_acc:.4f}, F1 Score: {val_f1:.4f}, ROC AUC: {val_roc_auc:.4f}")

        # Get teacher logits for outer training set
        teacher_logits_train = np.array([index_to_logits[idx] for idx in train_idx])

        # Preprocess the outer training and test sets
        X_train, X_test = preprocess(
            X_train,
            y_train,
            X_test,
            cat_cols,
            config,
            preprocessing_type="tree",
        )
        
        # Retrain CatBoost regressor on the full training set for the outer fold
        catboost = CatBoostRegressor(verbose=False)
        catboost.fit(X_train, teacher_logits_train)

        # Evaluate on the outer test set
        test_pred_logits = catboost.predict(X_test)
        # Convert predicted logits back to probabilities for evaluation
        test_probs = 1 / (1 + np.exp(-test_pred_logits))
        test_preds = (test_probs > 0.5).astype(int)
        
        test_acc = balanced_accuracy_score(y_test, test_preds)
        test_f1 = f1_score(y_test, test_preds, average="macro")
        test_roc_auc = roc_auc_score(y_test, test_probs)
        print(f"Outer Fold {outer_fold} - Test Accuracy: {test_acc:.4f}, Test F1 Score: {test_f1:.4f}, Test ROC AUC: {test_roc_auc:.4f}")
        
        # Save student probabilities with indices
        output_df = pd.concat(
            [
                output_df,
                pd.DataFrame({
                    "index": test_idx,
                    "output": test_probs  # Student probabilities (converted from predicted logits)
                })
            ],
            ignore_index=True
        )

    # Export the output DataFrame to a CSV file
    output_df = output_df.sort_values(by="index")
    output_file = os.path.join(config["data"]["cache_dir_path"], f"dataset_{dataset_id}_catboost_outputs_student.csv")
    output_df.to_csv(output_file, index=False)
    print(f"CatBoost student outputs saved to: {output_file}")

2025-06-04 09:02:28,598 - __main__ - INFO - Dataset 23381 loaded successfully from cache at 'data/cache/openml_dataset_23381.pkl'...
2025-06-04 09:02:28,887 - __main__ - INFO - Using GPU: NVIDIA RTX A6000
2025-06-04 09:02:28,967 - __main__ - INFO - Preprocessing pipeline for Decision Tree model.


Loaded fold indices from: data/cache/dataset_23381_fold_indices.json
Loaded TabPFN outputs from: data/cache/dataset_23381_tabpfn_outputs.csv


/home/mherre/miniconda3/envs/thesis/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [1, 2, 3] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
TBB Warning: The number of workers is currently limited to 11. The request for 39 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.

2025-06-04 09:02:35,085 - __main__ - INFO - Preprocessing pipeline for Decision Tree model.


Outer Fold 1, Inner Fold 1 - Accuracy: 0.6238, F1 Score: 0.6221, ROC AUC: 0.6975


Process LokyProcess-8:
Process LokyProcess-7:
Process LokyProcess-9:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/mherre/miniconda3/envs/thesis/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/mherre/miniconda3/envs/thesis/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/mherre/.local/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 478, in _process_worker
    _process_reference_size = _get_memory_usage(pid, force_gc=True)
  File "/home/mherre/.local/lib/python3.9/site-packages/joblib/externals/loky/process_executor.py", line 109, in _get_memory_usage
    gc.collect()
KeyboardInterrupt
  File "/home/mherre/miniconda3/envs/thesis/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/mherre/miniconda3/envs/thesis/lib/python3.9/multiprocessing/process.py", line 108,

KeyboardInterrupt: 

# Validation

In [ ]:
# Add this validation code after loading the fold indices and before the main loop
print("=== Validating Fold Indices ===")

# 1. Check that all indices are within valid range
total_samples = len(X)
print(f"Total samples in dataset: {total_samples}")

# 2. Collect all test indices across outer folds
all_test_indices = []
for fold_key, fold_data in fold_indices["outer_folds"].items():
    test_idx = np.array(fold_data["test_idx"])
    all_test_indices.extend(test_idx)

all_test_indices = np.array(all_test_indices)
print(f"Total test samples across all folds: {len(all_test_indices)}")
print(f"Unique test samples: {len(np.unique(all_test_indices))}")

# 3. Check if all samples appear exactly once in test sets
if len(all_test_indices) == len(np.unique(all_test_indices)) == total_samples:
    print("✓ All samples appear exactly once in test sets across folds")
else:
    print("✗ Issue with test set coverage!")

# 4. Check teacher outputs coverage
teacher_indices = set(teacher_outputs_df['index'].values)
dataset_indices = set(range(total_samples))
if teacher_indices == dataset_indices:
    print("✓ Teacher outputs cover all dataset indices")
else:
    missing = dataset_indices - teacher_indices
    extra = teacher_indices - dataset_indices
    print(f"✗ Teacher outputs mismatch - Missing: {missing}, Extra: {extra}")

# 5. Validate fold structure
for fold_key, fold_data in fold_indices["outer_folds"].items():
    outer_fold = int(fold_key.split('_')[1])
    train_idx = np.array(fold_data["train_idx"])
    test_idx = np.array(fold_data["test_idx"])
    
    # Check no overlap between train and test
    overlap = np.intersect1d(train_idx, test_idx)
    if len(overlap) == 0:
        print(f"✓ Fold {outer_fold}: No overlap between train/test")
    else:
        print(f"✗ Fold {outer_fold}: Found {len(overlap)} overlapping indices!")
    
    # Check inner folds
    inner_folds_data = fold_indices["inner_folds"][f"outer_fold_{outer_fold}"]
    inner_train_indices = []
    inner_val_indices = []
    
    for inner_fold_key, inner_fold_data in inner_folds_data.items():
        inner_train_idx = np.array(inner_fold_data["train_idx"])
        inner_val_idx = np.array(inner_fold_data["val_idx"])
        
        # Check inner indices are subset of outer train indices
        if np.all(np.isin(inner_train_idx, train_idx)) and np.all(np.isin(inner_val_idx, train_idx)):
            print(f"✓ Fold {outer_fold}, Inner fold {inner_fold_key}: Indices are subset of outer train")
        else:
            print(f"✗ Fold {outer_fold}, Inner fold {inner_fold_key}: Indices not subset of outer train!")
        
        inner_train_indices.extend(inner_train_idx)
        inner_val_indices.extend(inner_val_idx)
    
    # Check inner folds cover all outer training data
    inner_all = np.unique(np.concatenate([inner_train_indices, inner_val_indices]))
    if len(np.setdiff1d(train_idx, inner_all)) == 0:
        print(f"✓ Fold {outer_fold}: Inner folds cover all outer training data")
    else:
        missing_in_inner = np.setdiff1d(train_idx, inner_all)
        print(f"✗ Fold {outer_fold}: {len(missing_in_inner)} samples missing from inner folds")

print("=== End Validation ===\n")

=== Validating Fold Indices ===
Total samples in dataset: 500
Total test samples across all folds: 500
Unique test samples: 500
✓ All samples appear exactly once in test sets across folds
✓ Teacher outputs cover all dataset indices
✓ Fold 1: No overlap between train/test
✓ Fold 1, Inner fold inner_fold_1: Indices are subset of outer train
✓ Fold 1, Inner fold inner_fold_2: Indices are subset of outer train
✓ Fold 1: Inner folds cover all outer training data
✓ Fold 2: No overlap between train/test
✓ Fold 2, Inner fold inner_fold_1: Indices are subset of outer train
✓ Fold 2, Inner fold inner_fold_2: Indices are subset of outer train
✓ Fold 2: Inner folds cover all outer training data
✓ Fold 3: No overlap between train/test
✓ Fold 3, Inner fold inner_fold_1: Indices are subset of outer train
✓ Fold 3, Inner fold inner_fold_2: Indices are subset of outer train
✓ Fold 3: Inner folds cover all outer training data
✓ Fold 4: No overlap between train/test
✓ Fold 4, Inner fold inner_fold_1: Ind

In [ ]:
# Add this inside the main loop, after creating index_to_logits mapping
print(f"\n=== Validating Teacher Logits for Fold {outer_fold} ===")

# Check if all training indices have corresponding teacher outputs
missing_teacher_outputs = []
for idx in train_idx:
    if idx not in index_to_logits:
        missing_teacher_outputs.append(idx)

if len(missing_teacher_outputs) == 0:
    print(f"✓ All {len(train_idx)} training indices have teacher outputs")
else:
    print(f"✗ Missing teacher outputs for {len(missing_teacher_outputs)} indices: {missing_teacher_outputs[:10]}...")

# Check if test indices have teacher outputs (they shouldn't for proper validation)
test_in_teacher = []
for idx in test_idx:
    if idx in index_to_logits:
        test_in_teacher.append(idx)

if len(test_in_teacher) == 0:
    print(f"✓ No test indices found in teacher outputs (proper data leakage prevention)")
else:
    print(f"✗ WARNING: {len(test_in_teacher)} test indices found in teacher outputs - possible data leakage!")

# Sample a few logits to check they're reasonable
sample_indices = train_idx[:5]
sample_logits = [index_to_logits[idx] for idx in sample_indices]
sample_probs = 1 / (1 + np.exp(-np.array(sample_logits)))
print(f"Sample teacher logits: {sample_logits}")
print(f"Converted to probabilities: {sample_probs}")
print("=== End Teacher Logits Validation ===\n")


=== Validating Teacher Logits for Fold 5 ===
✓ All 400 training indices have teacher outputs
✗ WARNING: 100 test indices found in teacher outputs - possible data leakage!
Sample teacher logits: [-0.5883296581435397, -0.79494873035179, -0.49716914708297155, -0.6096161248627843, -0.4098342731792005]
Converted to probabilities: [0.3570182  0.31110707 0.37820616 0.35214677 0.39895186]
=== End Teacher Logits Validation ===



In [ ]:
# Add this at the very end to verify your student outputs match expectations
print("=== Final Validation ===")

# Check output file was created and has expected structure
if os.path.exists(output_file):
    student_df = pd.read_csv(output_file)
    print(f"✓ Student output file created with {len(student_df)} predictions")
    
    # Check all test indices are covered
    predicted_indices = set(student_df['index'].values)
    expected_test_indices = set(all_test_indices)
    
    if predicted_indices == expected_test_indices:
        print("✓ Student predictions cover exactly the expected test indices")
    else:
        missing = expected_test_indices - predicted_indices
        extra = predicted_indices - expected_test_indices
        print(f"✗ Prediction coverage mismatch - Missing: {len(missing)}, Extra: {len(extra)}")
    
    # Check output values are reasonable probabilities
    outputs = student_df['output'].values
    if np.all((outputs >= 0) & (outputs <= 1)):
        print(f"✓ All student outputs are valid probabilities [0,1]")
        print(f"  Range: [{outputs.min():.4f}, {outputs.max():.4f}]")
        print(f"  Mean: {outputs.mean():.4f}")
    else:
        invalid_count = np.sum((outputs < 0) | (outputs > 1))
        print(f"✗ {invalid_count} invalid probability values found!")
        
else:
    print("✗ Student output file was not created!")

print("=== End Final Validation ===")

=== Final Validation ===
✓ Student output file created with 500 predictions
✓ Student predictions cover exactly the expected test indices
✓ All student outputs are valid probabilities [0,1]
  Range: [0.2589, 0.7412]
  Mean: 0.4127
=== End Final Validation ===
